<a href="https://colab.research.google.com/github/marta-manzin/agentic-shopping-assistant/blob/main/agentic_frameworks_and_mcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
# Fill in the blank answers - delete these for class

# this class allows for lazy evaluation of variables, which I needed for one
# of the answers.
class _Answer:
    def __init__(self, var_name):
        self.var_name = var_name

    def _get_value(self):
        return globals()[self.var_name]

    def __repr__(self):
        return repr(self._get_value())

    def __str__(self):
        return str(self._get_value())

    def __getitem__(self, key):
        return self._get_value()[key]

    def __contains__(self, item):
        return item in self._get_value()

    def __iter__(self):
        return iter(self._get_value())

    def __getattr__(self, name):
        return getattr(self._get_value(), name)

__YOUR_ANSWER_HERE_1 = 'whats_in_the_cart'
__YOUR_ANSWER_HERE_2 = 'Get the contents of the shopping cart.'
__YOUR_ANSWER_HERE_3 = 'object'

__YOUR_ANSWER_HERE_4 = 'not in use yet -- to be used in call_tool'

__YOUR_ANSWER_HERE_5 = ['command']
__YOUR_ANSWER_HERE_6 = 0
__YOUR_ANSWER_HERE_7 = _Answer('event')


# Tools from the previous notebook

As part of [your homework]((https://colab.research.google.com/github/marta-manzin/agentic-shopping-assistant/blob/main/agentic_workshop_setup.ipynb)), you saved your OpenAI API key in a Colab Secret called "OPENAI_API_KEY". \
Now, import the key into the notebook.

In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

Start with a system prompt.

In [3]:
SYSTEM_PROMPT = """
You are a helpful assistant that adds and removes items from a shopping cart.

You have access to tools that let you:
1. Add grocery items
2. Remove grocery items
3. Inspect what is in the cart

Please do not ask me any follow-up questions after I make a request to you.
Just use the tools to satisfy my request.
"""

And with an empty cart.

In [4]:
MY_SHOPPING_CART = {}

Define the cart tools.

In [5]:
# @title add_to_cart
def add_to_cart(item_name: str, quantity: int=1) -> str:
  """Tool: "Add an grocery item to the shopping cart."."""
  try:
    if item_name not in MY_SHOPPING_CART:
      MY_SHOPPING_CART[item_name] = 0
    MY_SHOPPING_CART[item_name] += quantity
    return f"Added {quantity} {item_name}."
  except Exception as ex:
    return f"Failed to insert '{quantity}'. {ex!r}"

In [6]:
# @title remove_from_cart
def remove_from_cart(item_name: str) -> str:
  """Tool: Remove an item from the cart."""
  try:
    if item_name in MY_SHOPPING_CART:
      del MY_SHOPPING_CART[item_name]
      return f"Removed {item_name}."
    else:
      return f"{item_name} is not in the cart."
  except Exception as ex:
    return f"Failed to remove '{item_name}'. {ex!r}"

In [7]:
# @title whats_in_the_cart
def whats_in_the_cart() -> str:
  """Tool: Get the contents of the cart."""
  try:
    empty = True
    for item, quantity in MY_SHOPPING_CART.items():
      if quantity > 0:
        empty = False
        break
    if empty:
      return "The cart is empty."
    else:
      return f"Here's the cart: {dict(sorted(MY_SHOPPING_CART.items()))}"
  except Exception as ex:
    return f"Failed to get cart contents. {ex!r}"

# 🗄️ Creating an MCP Server

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/calling_tools.png" width="700">

</br>


</br>
</br>
</br>

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/mcp_tool_calling.png" width="700">

Create the MCP server.

In [8]:
%pip install --quiet mcp
from mcp.server import Server
from mcp.types import Tool, TextContent, Content

server = Server("set-server")
print("✓ Server created")

✓ Server created


Create an MCP wrapper for listing the available tools.

In [9]:
SERVER_TOOLS = [
  Tool(
      name="add_to_cart",
      description="Add a grocery item to the shopping cart.",
      inputSchema={
          "type": "object",
          "properties": {
              "item_name": {
                  "type": "string",
                  "description": "The grocery item to be added."
              },
              "quantity": {
                  "type": "integer",
                  "description": "The quantity of the grocery item to be added."
              },
          },
          "required": ["item_name"]
      }
    ),

    Tool(
        name="remove_from_cart",
        description="Remove an item from the cart.",
        inputSchema={
            "type": "object",
            "properties": {
                "item_name": {
                    "type": "string",
                    "description": "The grocery item to be removed."
                },
            },
            "required": ["item_name"]
        }
    ),

    Tool(
        name=__YOUR_ANSWER_HERE_1,
        description=__YOUR_ANSWER_HERE_2,
        inputSchema={
            "type": __YOUR_ANSWER_HERE_3,
            "properties": {},
            "required": []
        }
    )
]

In [10]:
async def list_tools() -> list[Tool]:
    """Return the list of available tools from our tools definition."""
    return SERVER_TOOLS

# Register the list_tools function with the server
server.list_tools()(list_tools)

<function __main__.list_tools() -> list[mcp.types.Tool]>

Create an MCP wrapper for executing tools.

In [11]:
async def call_tool(name: str, arguments: dict) -> list[Content]:
  """Handle MCP tool calls by delegating to our existing tools."""

  # Important! Verify that the function is one of the allowed tools
  allowed_tool_names = [t.name for t in SERVER_TOOLS]

  if name not in allowed_tool_names:
      return [TextContent(type="text", text=f"Error: '{name}' is not an allowed tool.")]

  tool_func = globals()[name]
  result = tool_func(**arguments)

  # Convert result to MCP response format
  return [TextContent(type="text", text=str(result))]

# Register the call_tool function with the server
server.call_tool()(call_tool)

<function __main__.call_tool(name: str, arguments: dict) -> list[mcp.types.TextContent | mcp.types.ImageContent | mcp.types.AudioContent | mcp.types.ResourceLink | mcp.types.EmbeddedResource]>

Create a web application.

In [12]:
# FastAPI is a framework for building REST APIs
%pip install --quiet fastapi
from mcp.server.sse import SseServerTransport
from fastapi import FastAPI, Request
from fastapi.responses import Response

# Create a FastAPI web application
app = FastAPI()

Expose a `POST` endpoint, used to handle incoming tool calls.

In [13]:
# Create an SSE transport that will handle messages at the "/messages" path
sse = SseServerTransport("/messages")

# Mount the POST handler for receiving messages
# Clients send messages to http://host:port/messages
app.mount("/messages", sse.handle_post_message)

Expose a `GET` endpoint, used to establish the connection to the server.

In [14]:
async def handle_sse(request: Request):
    """Handle incoming SSE connections from MCP clients."""
    # Connect the SSE transport to get read/write streams
    async with sse.connect_sse(
        request.scope, request.receive, request._send
    ) as (read_stream, write_stream):
        # Run the MCP server with these streams
        await server.run(
            read_stream,
            write_stream,
            server.create_initialization_options()
        )
    return Response()

# Register the GET endpoint with the FastAPI app
# Clients connect to http://host:port/sse to establish SSE connection
app.add_api_route("/sse", handle_sse, methods=["GET"])

### Set up to start and stop the server

In [15]:
# @title
# Uvicorn is a web server that handles HTTP requests and asynchronous code
%pip install --quiet uvicorn psutil

import uvicorn, time, os, psutil
from multiprocessing import Process


def get_pid_on_port(port):
    '''Utility method to get the process ID running at a given port.'''
    for conn in psutil.net_connections(kind='inet'):
        if conn.laddr.port == port and conn.status == 'LISTEN':
            if conn.pid != os.getpid():
                return conn.pid
    return None


def start_server(app_obj, port):
    '''Idempotent function to start the server at a given port.'''
    pid = get_pid_on_port(port)
    if pid:
        print(f"✗ Server already running at http://127.0.0.1:{port}")
        return

    p = Process(target=uvicorn.run, args=(app_obj,),
                kwargs={'host': "0.0.0.0", 'port': port, 'log_level': "warning"},
                daemon=True)
    p.start()
    time.sleep(3)
    if p.is_alive():
        print(f"✓ Server started at http://127.0.0.1:{port}")
    else:
        print("✗ Server failed to start.")


def stop_server(port):
    '''Idempotent function to stop the server at a given port.'''
    pid = get_pid_on_port(port)
    if not pid:
        print(f"✗ No server running at http://127.0.0.1:{port}.")
        return
    try:
        proc = psutil.Process(pid)
        proc.terminate()
        proc.wait(timeout=3)
        print("✓ Server stopped.")
    except psutil.TimeoutExpired:
        print("✗ Server is taking too long to exit.")



### Start the web server.

In [16]:
server_port = 12345
stop_server(server_port) # Make sure that no server is running
start_server(app, server_port)

server_url = f"http://127.0.0.1:{server_port}/sse"

✗ No server running at http://127.0.0.1:12345.
✓ Server started at http://127.0.0.1:12345


# 🤝 Creating an MCP Client

List available tools on the MCP server.

In [17]:
from mcp import ClientSession
from mcp.client.sse import sse_client

# Utility function to list tools
async def list_mcp_tools():
    """Helper function to list available MCP tools."""
    async with sse_client(server_url) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            available_tools = await session.list_tools()
            return available_tools

available_tools = await list_mcp_tools()
print("Available tools:", [t.name for t in available_tools.tools])

Available tools: ['add_to_cart', 'remove_from_cart', 'whats_in_the_cart']


Test each tool, starting with an empty set.

In [18]:
# Utility function to call a tool
async def call_mcp_tool(tool_name: str, arguments: dict = {}):
    """Helper function to call an MCP tool."""
    async with sse_client(server_url) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            result = await session.call_tool(tool_name, arguments)
            return result

Add "cherries" to the cart.

In [19]:
result = await call_mcp_tool("add_to_cart", {"item_name": "cherries"})
print(result.content[0].text)

Added 1 cherries.


Attempt to remove "bananas" from the set.

In [20]:
result = await call_mcp_tool("remove_from_cart", {"item_name": "bananas"})
print(result.content[0].text)

bananas is not in the cart.


Read all contents of the set.

In [21]:
result = await call_mcp_tool("whats_in_the_cart")
print(result.content[0].text)

Here's the cart: {'cherries': 1}


# 🧠 Orchestration with LangGraph

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/langchain_ecosystem.png" width="700">

^: *[From LangChain vs LangGraph vs LangSmith vs LangFlow: Understanding through a Realtime Project](https://aws.plainenglish.io/langchain-vs-langgraph-vs-langsmith-vs-langflow-understanding-through-a-realtime-project-2c3efd1606e7)*

In [22]:
%pip install --quiet --upgrade "langchain" "langchain-openai" "langgraph" "langchain-mcp-adapters"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.3/496.3 kB 27.3 MB/s eta 0:00:00


In [23]:
from langchain_mcp_adapters.client import MultiServerMCPClient

# Create MCP client that connects to your set-server
client = MultiServerMCPClient(
    {
        "set-server": {
            "transport": "sse",
            "url": f"http://localhost:{server_port}/sse",
        }
    }
)

Get available tools from the MCP server.

In [24]:
tools_from_mcp = await client.get_tools()

for tool in tools_from_mcp:
    print(f"- {tool.name}: {tool.description}")

- add_to_cart: Add a grocery item to the shopping cart.
- remove_from_cart: Remove an item from the cart.
- whats_in_the_cart: Get the contents of the shopping cart.


Create a LangGraph agent.

In [25]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

agent_executor = create_agent(
    ChatOpenAI(model="gpt-4o", temperature=0),
    tools_from_mcp,
)

The following function calls the LangGraph agent and prints the conversation history.

In [26]:
async def submit_langgraph_request(user_prompt: str, verbose: bool = True):
    """Submit a request to the LangGraph agent and display the conversation."""
    from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

    # Run the agent with the system prompt and user's prompt
    result = await agent_executor.ainvoke({
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
    })

    # Display the conversation if verbose
    if verbose:
        for message in result['messages']:

            # Print user message
            if isinstance(message, HumanMessage):
                print("\n👤 User: " + message.content)

            # Print agent message
            elif isinstance(message, AIMessage):
                if message.content:
                    print("\n🤖 Agent: " + message.content)

            # Print tool action
            elif isinstance(message, ToolMessage):
                # Extract text from content (handle both string and list of dicts)
                if isinstance(message.content, str):
                    outcome_text = message.content
                elif isinstance(message.content, list) and len(message.content) > 0:
                    # Extract 'text' field from the first item if it's a dict
                    outcome_text = message.content[0].get('text', str(message.content[0]))
                else:
                    outcome_text = str(message.content)

                print(f"\n🔧 {message.name}: {outcome_text}")

Let's test it!

In [27]:
await submit_langgraph_request("""
  Please add all the ingredients for lasagna into the cart.""")


👤 User: 
  Please add all the ingredients for lasagna into the cart.

🤖 Agent: To add all the ingredients for lasagna into the cart, I'll include the following common ingredients:

1. Lasagna noodles
2. Ground beef
3. Ricotta cheese
4. Mozzarella cheese
5. Parmesan cheese
6. Marinara sauce
7. Onion
8. Garlic
9. Olive oil
10. Salt
11. Black pepper
12. Basil
13. Oregano

I'll add these items to the cart now.

🔧 add_to_cart: Added 1 Lasagna noodles.

🔧 add_to_cart: Added 1 Ground beef.

🔧 add_to_cart: Added 1 Ricotta cheese.

🔧 add_to_cart: Added 1 Mozzarella cheese.

🔧 add_to_cart: Added 1 Parmesan cheese.

🔧 add_to_cart: Added 1 Marinara sauce.

🔧 add_to_cart: Added 1 Onion.

🔧 add_to_cart: Added 1 Garlic.

🔧 add_to_cart: Added 1 Olive oil.

🔧 add_to_cart: Added 1 Salt.

🔧 add_to_cart: Added 1 Black pepper.

🔧 add_to_cart: Added 1 Basil.

🔧 add_to_cart: Added 1 Oregano.

🤖 Agent: All the ingredients for lasagna have been added to the cart:

- Lasagna noodles
- Ground beef
- Ricotta che

In [28]:
await submit_langgraph_request("""
  I would rather shop for a greek dinner.
  Can you fill up my shopping cart with the
  ingredients for Moussaka and take out all that
  Italian stuff that I don't need.""")


👤 User: 
  I would rather shop for a greek dinner.
  Can you fill up my shopping cart with the
  ingredients for Moussaka and take out all that
  Italian stuff that I don't need.

🔧 whats_in_the_cart: Here's the cart: {'Basil': 1, 'Black pepper': 1, 'Garlic': 1, 'Ground beef': 1, 'Lasagna noodles': 1, 'Marinara sauce': 1, 'Mozzarella cheese': 1, 'Olive oil': 1, 'Onion': 1, 'Oregano': 1, 'Parmesan cheese': 1, 'Ricotta cheese': 1, 'Salt': 1, 'cherries': 1}

🔧 remove_from_cart: Removed Basil.

🔧 remove_from_cart: Removed Lasagna noodles.

🔧 remove_from_cart: Removed Marinara sauce.

🔧 remove_from_cart: Removed Mozzarella cheese.

🔧 remove_from_cart: Removed Oregano.

🔧 remove_from_cart: Removed Parmesan cheese.

🔧 remove_from_cart: Removed Ricotta cheese.

🔧 add_to_cart: Added 2 Eggplant.

🔧 add_to_cart: Added 2 Potatoes.

🔧 add_to_cart: Added 1 Tomato paste.

🔧 add_to_cart: Added 1 Red wine.

🔧 add_to_cart: Added 1 Cinnamon.

🔧 add_to_cart: Added 1 Allspice.

🔧 add_to_cart: Added 1 Nutm

In [29]:
await submit_langgraph_request("""
  I'm American. How about just a hot dog with mustard.
  Can you take out all those other ingredients too.""")


👤 User: 
  I'm American. How about just a hot dog with mustard.
  Can you take out all those other ingredients too.

🔧 whats_in_the_cart: Here's the cart: {'Allspice': 1, 'Black pepper': 1, 'Cinnamon': 1, 'Eggplant': 2, 'Eggs': 2, 'Feta cheese': 1, 'Flour': 1, 'Garlic': 1, 'Ground beef': 1, 'Milk': 1, 'Nutmeg': 1, 'Olive oil': 1, 'Onion': 1, 'Potatoes': 2, 'Red wine': 1, 'Salt': 1, 'Tomato paste': 1, 'cherries': 1}

🔧 remove_from_cart: Removed Allspice.

🔧 remove_from_cart: Removed Black pepper.

🔧 remove_from_cart: Removed Cinnamon.

🔧 remove_from_cart: Removed Eggplant.

🔧 remove_from_cart: Removed Eggs.

🔧 remove_from_cart: Removed Feta cheese.

🔧 remove_from_cart: Removed Flour.

🔧 remove_from_cart: Removed Garlic.

🔧 remove_from_cart: Removed Ground beef.

🔧 remove_from_cart: Removed Milk.

🔧 remove_from_cart: Removed Nutmeg.

🔧 remove_from_cart: Removed Olive oil.

🔧 remove_from_cart: Removed Onion.

🔧 remove_from_cart: Removed Potatoes.

🔧 remove_from_cart: Removed Red wine.

🔧

# 👩🏻‍💻 Creating a Coding Assistant

Here's what the outdated application looks like:
https://dwoodlock.github.io/Metric-Treadmill-2017/

## Set up the System Prompt(s)

In [30]:
system_prompt = """
  You are a helpful assistant that can interact with a computer using tools.

  CRITICAL REQUIREMENT: You MUST ALWAYS provide your reasoning BEFORE
  calling any tool.

  For EVERY action you take:
  1. First, write a THOUGHT section in your text response explaining what
  you plan to do and why
  2. Then, after your THOUGHT, make the tool call

  NEVER call a tool without first explaining your reasoning in text.
  The content field of your response must NEVER be empty.

  Example:
  THOUGHT: I need to see what files exist in the directory to understand the
   codebase structure.
  [then call bash_tool with ls command]
  """

The user prompt also contains instructions

In [31]:
# @title User Prompt Template
import platform

user_prompt_template = """
Please complete this task (bug fix, modernization, enhancement, or new application):

{{user-prompt}}

You can use the bash tool to execute commands and manipulate files to implement the necessary changes.

Task types you may encounter:
- Bug fix: Identify and resolve a defect.
- Modernization: Update outdated dependencies, APIs, build tooling, or code patterns.
- Enhancement: Add or improve functionality, performance, or UX.
- New application: Scaffold a fresh project and deliver a minimal, runnable MVP with docs/tests.

## Recommended Workflow

This workflow should be done step-by-step so that you can iterate on your changes and any possible problems. Determine the task type first and follow the relevant branch below.

1. Analyze the codebase by finding and reading relevant files
2. Define the task type (Bug fix / Modernization / Enhancement / New application) and acceptance criteria
3. If Bug fix:
   a. Reproduce the issue or create a minimal reproduction script
   b. Identify the root cause in the code
   c. Edit the source code to resolve the issue
   d. Verify the fix by running the reproduction again
4. If Modernization:
   a. Inventory outdated dependencies, APIs, build tooling, and patterns
   b. Propose a minimal, safe modernization plan (scoped to this task)
   c. Apply updates incrementally (dependencies, code changes, config/build)
   d. Ensure backward compatibility or provide migration notes as needed
5. If Enhancement:
   a. Specify the new or improved behavior (UI/API/contracts)
   b. Implement the change with clear, maintainable code and tests
   c. Update documentation or usage examples if applicable
6. If New application:
   a. Confirm requirements, scope, target runtime, and preferred stack
   b. Scaffold project structure and tooling (venv/package manager, lint/format, tests, CI optional)
   c. Initialize minimal dependencies with pinned versions
   d. Implement MVP feature(s) with clear entrypoint(s) (CLI/HTTP/UI)
   e. Provide run/build/test commands and ensure the app starts cleanly
   f. Add minimal docs: README with setup, usage, architecture, and key decisions
   g. Add basic tests and sample config/data as needed
   h. Package or containerize if appropriate (optional)
7. Test edge cases, performance implications, and regression risks
8. Clean up all temporary files you created (test scripts, backup files, patch files, etc.) and terminate any background processes you started to leave the repository clean
9. Submit your changes and finish your work by calling the bash tool with the command: `echo COMPLETE_TASK_AND_SUBMIT_FINAL_OUTPUT`.
   Do not combine it with any other command. <important>After this command, you cannot continue working on this task.</important>

## Important Rules

1. Directory or environment variable changes are not persistent. Every bash command is executed in a new subshell.
   However, you can prefix any command with `MY_ENV_VAR=MY_VALUE cd /path/to/working/dir && ...` or write/load environment variables from files

## Testing Constraints and Patterns

**CRITICAL:** The bash shell is non-interactive. Commands that block
waiting for input or running servers in the foreground WILL HANG
INDEFINITELY. You must manage background processes explicitly.

1. **NEVER run long-lived servers in the foreground** - they will
   hang the shell. Always use `&` to background them.
2. If you must start a server to test endpoints:
   - Start it in the background with `&` and capture `$!` as the PID
   - Probe for readiness using retries with timeouts
   - Run your test assertions (curl, wget, etc.)
   - Kill the process and wait for cleanup
3. Prefer unit tests over integration tests when possible to avoid
   server management complexity.
4. Use environment variables for PORTs; default to 3000 when
   unstated.
5. Always terminate background processes during cleanup (step 8).

Example pattern for HTTP server testing:

```bash
# 1) Choose a port and start server in background
PORT=${PORT:-3000}
cd path/to/app || exit 1
python3 -m http.server "$PORT" >/tmp/app_server.log 2>&1 &
SERVER_PID=$!

# 2) Wait for readiness with timeout (10s) and retries
READY=0
for i in $(seq 1 20); do
  curl --silent --fail --max-time 1 \
    "http://localhost:$PORT/" >/dev/null && { READY=1; break; }
  sleep 0.5
done
[ "$READY" -eq 1 ] || {
  echo "Server failed to start";
  kill $SERVER_PID || true;
  exit 1;
}

# 3) Run test assertions
curl --silent --fail "http://localhost:$PORT/" | grep "expected"
EXIT_CODE=$?

# 4) ALWAYS teardown - kill server and wait for cleanup
kill $SERVER_PID
wait $SERVER_PID 2>/dev/null || true

exit $EXIT_CODE
```

Static apps: Prefer direct file checks (HTML/JS/CSS) or serve via
a lightweight server as above. For unit tests, run the project's
test runner without starting a server if possible.

<system_information>
 {{platform_info}}
</system_information>
""".replace("{{platform_info}}", str(platform.uname()))

## Update the Tools - Add the Bash Tool

In [32]:
def bash_tool(command: str) -> str:
    """Execute a bash command and return the result."""
    import subprocess  # Import inside function for multiprocessing compatibility
    import os

    # Get CODE_DIRECTORY from environment variable, with fallback
    code_directory = os.environ.get("CODE_DIRECTORY", "./Metric-Treadmill-2017")

    if not command.isascii():
        return "Error: Command contains non-ASCII characters."
    try:
        result = subprocess.run(
            command, shell=True, check=True,
            capture_output=True, text=False,  # Get bytes first
            cwd=code_directory
        )

        # Try to decode as UTF-8
        try:
            output = result.stdout.decode('utf-8')
        except UnicodeDecodeError:
            # Binary data detected - provide helpful error
            return (
                "Error: Command output contains binary data (non-text file). "
                "Try filtering to text files only, e.g.:\n"
                "- Add file type filters: -name '*.css' -o -name '*.js'\n"
                "- Or use: file <filename> | grep -q text && cat <filename>"
            )

        return output if output else (
            "Command executed successfully with no output."
        )
    except subprocess.CalledProcessError as e:
        # Decode stderr, handling potential binary data there too
        try:
            stderr = e.stderr.decode('utf-8') if e.stderr else str(e)
        except UnicodeDecodeError:
            stderr = "Error output contains binary data"
        return f"Error executing command: {stderr}"

bash_description = """
Execute bash commands on the system.

## Useful command examples

### Create a new file:
cat <<'EOF' > newfile.py
import numpy as np
hello = "world"
print(hello)
EOF

### Edit files with sed:

# Replace all occurrences
sed -i 's/old_string/new_string/g' filename.py

# Replace only first occurrence
sed -i 's/old_string/new_string/' filename.py

# Replace first occurrence on line 1
sed -i '1s/old_string/new_string/' filename.py

# Replace all occurrences in lines 1-10
sed -i '1,10s/old_string/new_string/g' filename.py

### View file content:
# View specific lines with numbers
nl -ba filename.py | sed -n '10,20p'

### Any other command you want to run
You can run any bash command including ls, cat, find, python, etc.
""".strip()

In [33]:
# Add the Bash tool to our server tools list.  (Check if it's been added already in case the cell is re-run)
from mcp.types import Tool
if 'bash_tool' not in [tool.name for tool in SERVER_TOOLS]:
    SERVER_TOOLS.append(
      Tool(
        name="bash_tool",
        description=bash_description,
        inputSchema={
            "type": "object",
            "properties": {
                "command": {
                    "type": "string",
                    "description": "The bash command to execute."
                }
            },
            "required": __YOUR_ANSWER_HERE_5
        }
      )
    )


In [34]:
# need to stop the server and restart it so it sees the new tools
stop_server(server_port)
start_server(app, server_port)

[tool.name for tool in await client.get_tools()]

✓ Server stopped.
✓ Server started at http://127.0.0.1:12345


['add_to_cart', 'remove_from_cart', 'whats_in_the_cart', 'bash_tool']

## Get the User Task

In [35]:
user_prompt = """
I have a web app that I wrote many years ago that converts my running plans
into a metric system if I find myself on a treadmill internationally.
I’m concerned that it uses old libraries and old approaches and it’ll just
stop working one day.  Can you modernize this app for me.  I'd especially
like you to eliminate unneeded and deprecated libraries and use modern
language features and approaches.
""".strip()

In [36]:
# Insert the user request into the user prompt template to create the full prompt
enhanced_user_prompt = user_prompt_template.replace(
    "{{user-prompt}}",
    user_prompt
)

## Get the Code

In [37]:
CODE_DIRECTORY = "./Metric-Treadmill-2017"

In [38]:
# Clone the Metric-Treadmill-2017 repo
from pathlib import Path
import shutil
import subprocess

path = Path("./Metric-Treadmill-2017")
if path.exists() and path.is_dir(): shutil.rmtree(path)

subprocess.run(["git", "clone", "--quiet", "https://github.com/dwoodlock/Metric-Treadmill-2017.git"],
               check=True)

CompletedProcess(args=['git', 'clone', '--quiet', 'https://github.com/dwoodlock/Metric-Treadmill-2017.git'], returncode=0)

## Set up a new LangGraph agent

In [39]:
tools_from_mcp = await client.get_tools()

In [40]:
# Create a LangGraph agent using the updated tools and higher-end model
from langchain.agents import create_agent

my_coding_agent = create_agent(
    ChatOpenAI(model="gpt-5.2-2025-12-11", temperature=__YOUR_ANSWER_HERE_6),
    tools_from_mcp,
)

print("✓ LangGraph coding agent created")

✓ LangGraph coding agent created


In [41]:
# get initial message ready (the agent already knows about the tools from above)

initial_message = {
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": enhanced_user_prompt}]}

In [42]:
# here's the function to step through the events and print out each one as it's happening

async def run_coding_agent(user_prompt):

    iteration = 0

    # a little code repeated from above - Sorry!
    enhanced_user_prompt = user_prompt_template.replace(
        "{{user-prompt}}", user_prompt)

    initial_message = {"messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": enhanced_user_prompt}]}

    async for event in my_coding_agent.astream(initial_message):
        iteration = iteration + 1
        print("-" * 40, "\nIteration", iteration)
        print_event(event)

    return event

In [43]:
# A helper function to print out the details of the event

from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
import textwrap

def print_event(event):
    wrap_width = 120

    if 'model' in event:
        messages = event['model']['messages']
    elif 'tools' in event:
        messages = event['tools']['messages']
    else:
        assert False, "Unimplemented event structure"

    for message in messages:
        if isinstance(message, AIMessage):
            if type(message.content) == str:
                wrapped = textwrap.fill(message.content, width=wrap_width,
                                       initial_indent="AI: \033[34m",
                                       subsequent_indent="    ")
                print(wrapped + "\033[0m")
            elif type(message.content) == list:
                for content_block in message.content:
                    if content_block['type'] == 'tool_use':
                        call_str = f"AI: call {content_block['name']} {content_block['input']}"
                        wrapped = textwrap.fill(call_str, width=wrap_width, subsequent_indent="    ")
                        print("\033[34m" + wrapped + "\033[0m")
                    elif content_block['type'] == 'text':
                        wrapped = textwrap.fill(content_block['text'], width=wrap_width,
                                               initial_indent="AI: \033[34m ",
                                               subsequent_indent="    ")
                        print(wrapped + "\033[0m")
            else:
                assert False, "Unimplemented message.content type"
        elif isinstance(message, ToolMessage):
            if type(message.content) == list:
                for content_block in message.content:
                    if content_block['type'] == 'text':
                        wrapped = textwrap.fill(content_block['text'], width=wrap_width,
                                               initial_indent="TOOL RESULT:\n\033[32m",
                                               subsequent_indent="")
                        if len(wrapped) > 420:
                          print(wrapped[0:420] + " ... " + "\033[0m")
                        else:
                          print(wrapped + "\033[0m")
                    else:
                        assert False, "Unimplemented message.content type"
            else:
                assert False, "Unimplemented message.content type"
        else:
            assert False, "Unimplemented message type"

        if hasattr(message, 'tool_calls'):
            for tool_call in message.tool_calls:
                call_str = f"AI: TOOL CALL REQUEST: {tool_call['name']} {tool_call['args']}"
                wrapped = textwrap.fill(call_str, width=wrap_width, subsequent_indent="    ")
                if len(wrapped) > 420:
                  print(wrapped[0:420] + " ... ")
                else:
                  print(wrapped)

In [44]:
# before we run the whole loop, let's take a look at just one iteration

event = await anext(my_coding_agent.astream(initial_message))
event

{'model': {'messages': [AIMessage(content='THOUGHT: I’ll first inspect the repository structure and identify the current stack (static HTML/JS, Node, Python, etc.) and dependencies so I can propose a safe modernization plan and then implement it incrementally.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 1833, 'total_tokens': 1905, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1536}}, 'model_provider': 'openai', 'model_name': 'gpt-5.2-2025-12-11', 'system_fingerprint': None, 'id': 'chatcmpl-D6Id4VmK3bMCLlewYxWiRMCiXkbjf', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c33b3-d859-7eb0-b098-e1d1286b680a-0', tool_calls=[{'name': 'bash_tool', 'args': {'command': 'ls -ლა'}, 'id': 'call_jluzqVgFL0JRplwgiiU8maNT', 'type': 'too

In [45]:
# slightly prettier form
print_event(__YOUR_ANSWER_HERE_7)

AI: THOUGHT: I’ll first inspect the repository structure and identify the current stack (static HTML/JS, Node,
    Python, etc.) and dependencies so I can propose a safe modernization plan and then implement it incrementally.
AI: TOOL CALL REQUEST: bash_tool {'command': 'ls -ლა'}


In [46]:
print("Here we go!")

Here we go!


In [47]:
result = await run_coding_agent(user_prompt)

---------------------------------------- 
Iteration 1
AI: THOUGHT: I’ll first inspect the repository to understand the app’s current structure, language stack, and
    dependencies so I can identify outdated/deprecated libraries and plan a safe modernization path.
AI: TOOL CALL REQUEST: bash_tool {'command': 'ls -la'}
---------------------------------------- 
Iteration 2
TOOL RESULT:
total 112 drwxr-xr-x 5 root root  4096 Feb  6 16:04 . drwxr-xr-x 1 root root  4096 Feb  6 16:04 ..
-rw-r--r-- 1 root root 76553 Feb  6 16:04 agentic_shopping_assistant.ipynb -rw-r--r-- 1 root root  5204 Feb  6 16:04
app.js drwxr-xr-x 8 root root  4096 Feb  6 16:04 .git -rw-r--r-- 1 root root    11 Feb  6 16:04 .gitignore -rw-r--r-- 1
root root  1583 Feb  6 16:04 index.html drwxr-xr-x 4 root root  40 ... 
---------------------------------------- 
Iteration 3
AI: THOUGHT: Next I’ll open the HTML/JS to see what libraries are used (likely old jQuery Mobile) and how
    conversions are implemented, then decide 

## Let's take a look at the new site!

In [48]:
# @title Start a Simple Web Server
import os
import time
from IPython.display import display, HTML
from random import randint
import subprocess
from google.colab.output import eval_js

query_param = f"?v={randint(10000, 99999)}"

# Kill ANY process using port 8000
!fuser -k 8000/tcp 2>/dev/null || true

# Wait a moment for port to be released
time.sleep(1)

# Start server in background
proc = subprocess.Popen(
    ['python', '-m', 'http.server', '8000', '--directory', 'Metric-Treadmill-2017'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for server to start
time.sleep(2)

# Check if process is still running
if proc.poll() is None:
    print("Server started successfully!")
    # Get the proxy URL
    base_url = eval_js("google.colab.kernel.proxyPort(8000)")
    full_url = f"{base_url}{query_param}"
    display(HTML(f'<a href="{full_url}" target="_blank" style="font-size: 18px;">Click here to view your website</a>'))
else:
    print("Server failed to start!")
    stdout, stderr = proc.communicate()
    print("STDOUT:", stdout.decode())
    print("STDERR:", stderr.decode())

Server started successfully!


In [ ]:
# download the resulting website so you can take a look.

!zip -q -r -9 metric-treadmill.zip ./Metric-Treadmill-2017 \
  -x "**/.git/**" "**/node_modules/**" "**/.DS_Store"

from google.colab import files
files.download("metric-treadmill.zip")

# 🧹 MCP Cleanup

Stop the MCP server.

In [ ]:
# Stop the MCP server using the ServerManager
# stop_server(server_port)

# Thank you!

Please share your feedback through [this anonymous survey](https://forms.cloud.microsoft/r/SVPFgQbWQJ).